In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

In [5]:
img1 = cv.imread('./detect/book.jpg', cv.IMREAD_GRAYSCALE)
print(type(img1))

<class 'numpy.ndarray'>


In [7]:

 
MIN_MATCH_COUNT = 10
 
img1 = cv.imread('./detect/book.jpg', cv.IMREAD_GRAYSCALE)          # queryImage
img2 = cv.imread('./detect/book_on_table.jpg', cv.IMREAD_GRAYSCALE) # trainImage
 
# Initiate SIFT detector
sift = cv.SIFT_create()
 
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
 
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
 
flann = cv.FlannBasedMatcher(index_params, search_params)
 
matches = flann.knnMatch(des1,des2,k=2)
 
# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

In [9]:
print(type(kp1), type(des1))

<class 'tuple'> <class 'numpy.ndarray'>


In [ ]:
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
 
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
 
    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts,M)
 
    img2 = cv.polylines(img2,[np.int32(dst)],True,255,3, cv.LINE_AA)
 
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None

In [ ]:
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
 
img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
 
plt.imshow(img3, 'gray'),plt.show()

In [ ]:
cap = cv.VideoCapture("./detect/tea_video.mp4")
backSub = cv.createBackgroundSubtractorMOG2()
if not cap.isOpened():
    print("Error opening video file")
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    #print(ret, frame)
    if ret:
        # Apply background subtraction
        fg_mask = backSub.apply(frame)
contours, hierarchy = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# print(contours)
frame_ct = cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)
# Display the resulting frame
cv2.imshow('Frame_final', frame_ct)


In [ ]:
cv.destroyAllWindows()

In [ ]:
with np.load('CameraParams.npz') as file:
    mtx, dist, rvecs, tvecs = [file[i] for i in ('cameraMatrix', 'dist', 'rvecs', 'tvecs')]

In [ ]:
images = glob.glob('./detect/book.jpg')
print(images)
ind = 0
for fname in images:

    ind += 1
    img = cv.imread(fname)
    h, w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    # undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imwrite('./detect/book_'+str(ind)+'.png', img)

In [ ]:
cap = cv.VideoCapture("./detect/tea_video.mp4")
if not cap.isOpened():
    print("Error opening video file")

ind = 0
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if frame is None:
        break
    h, w = frame.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    # undistort
    dst = cv.undistort(frame, mtx, dist, None, newcameramtx)
    cv.imwrite('./videoframes_tea/frame_'+str(ind)+'.png', frame)
    ind += 1

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
 
MIN_MATCH_COUNT = 10
 
img1 = cv.imread('./detect/new_book.jpg', cv.IMREAD_GRAYSCALE)          # queryImage
sift = cv.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1,None)


def findMatch(fname, out):
    global kp1, des1;
    imgS = cv.imread(fname) # trainImage
    img2 = cv.cvtColor(imgS, cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp2, des2 = sift.detectAndCompute(img2,None)
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv.FlannBasedMatcher(index_params, search_params)
 
    matches = flann.knnMatch(des1,des2,k=2)
 
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.6*n.distance:
            good.append(m)

    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
 
        M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        #drawCountour(img, M)
 
        h,w = img1.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
        pts = np.float32([ [123,1146],[2772,2175],[3484,720],[1209,263]]).reshape(-1,1,2)
        pts2 = np.float32([[207,1402],[2748,2465],[3455,983],[1267,485]]).reshape(-1,1,2)
        pts3 = np.float32([[123,1146],[207,1402],[2748,2465],[2772,2175],[3484,720],[3455,983],[1267,485],[1209,263]]).reshape(-1,1,2)
        #pts = np.float32([ [133,1159],[2769, 2176],[3487, 718],[1214,264], [133,1159], [266,1410], [2734,2425],[3397,1029], [1347,515], [266,1410], [1347,515], [1214,264],[3487, 718],[3397,1029], [2734,2425],[2769, 2176]]).reshape(-1,1,2)
        dst = cv.perspectiveTransform(pts,M)
        dst2 = cv.perspectiveTransform(pts2,M)
        dst3 = cv.perspectiveTransform(pts3,M)
        img2 = cv.polylines(imgS,[np.int32(dst)],True, (0, 255, 0), 5)
        img2 = cv.polylines(img2,[np.int32(dst2)],True, (0, 255, 0), 5)
        img2 = cv.polylines(img2,[np.int32(dst3)],True, (0, 255, 0), 5)
        for i, m in enumerate(good):
            pt = (int(dst_pts[i][0][0]), int(dst_pts[i][0][1]))
            if matchesMask[i]:
                cv.circle(img2, pt, 5, (0, 255, 255), -1)  # Inliers желтые
            else:
                cv.circle(img2, pt, 5, (0, 0, 255), -1)    # Outliers красные
 
    else:
        print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
        matchesMask = None
    
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
    singlePointColor = None,
    matchesMask = matchesMask, # draw only inliers
    flags = 2)
    #img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
    #plt.imshow(img3),plt.show()
    #img3 = cv.drawKeypoints(img2, [kp2[m.trainIdx] for m in good], cv.DRAW_MATCHES_FLAGS_DEFAULT, color=(0, 255, 255))

    #plt.imshow(img2),plt.show()
    out.write(img2)
    cv.imwrite('./videoframes_rotate_detected/frame_'+str(ind)+'.png', img2)
    plt.imshow(img2),plt.show()

In [ ]:
def trans(points_r, rvecs, tvec, cam_m, dist):
    p_2d = []
    for i in range(len(points_r)):
        x = points_r[i][0]
        y = points_r[i][1]
        z = points_r[i][2]
        p_t = np.zeros(3)
        p_t[0] = rvecs[0][0]*x + rvecs[0][1]*y + rvecs[0][2]*z + tvec[0]
        p_t[1] = rvecs[1][0]*x + rvecs[1][1]*y + rvecs[1][2]*z + tvec[1]
        p_t[2] = rvecs[2][0]*x + rvecs[2][1]*y + rvecs[2][2]*z + tvec[2]
        x_ = p_t[0]/p_t[2]
        y_ = p_t[1]/p_t[2]
        r_2 = (x_)**2 + (y_)**2
        x_d = x_ * (1 + dist[0]*r_2 + dist[1]*(r_2)**2 + dist[2]*(r_2)**3)/(1 + dist[3]*r_2 + dist[3]*(r_2)**2)
        y_d = y_ * (1 + dist[0]*r_2 + dist[1]*(r_2)**2 + dist[2]*(r_2)**3)/(1 + dist[3]*r_2 + dist[3]*(r_2)**2)
        u = cam_m[0][0]*x_d + cam_m[0][2]
        v = cam_m[1][1] * y_d + cam_m[1][2]
        p_2d.append([u, v])
        
    return np.array(p_2d)

In [ ]:
def trans(points_r, rvecs, tvec, cam_m, dist):
    p_2d = np.zeros((len(points_r), 2))
    dist = np.pad(dist[0], (0, 6 - len(dist[0])), 'constant', constant_values=0)
    for i in range(len(points_r)):
        p_t = (rvecs @ points_r[i] + tvec.T)[0]
        x_ = p_t[0]/p_t[2]
        y_ = p_t[1]/p_t[2]
        r_2 = np.power(x_, 2) + np.power(y_, 2)
        coeff =  (1 + dist[0]*r_2 + dist[1]*np.power(r_2, 2) + dist[2]*np.power(r_2, 3))/(1 + dist[3]*r_2 + dist[4]*np.power(r_2, 2) + dist[5]*np.power(r_2, 3))
        u = cam_m[0][0] * x_ * coeff + cam_m[0][2]
        v = cam_m[1][1] * y_ * coeff + cam_m[1][2]
        p_2d[i][0] = u
        p_2d[i][1] = v
        
    return p_2d

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
 
MIN_MATCH_COUNT = 10
Z_LENGTH = 881.77
 
img1 = cv.imread('./detect/tea.jpg', cv.IMREAD_GRAYSCALE)          # queryImage
sift = cv.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1,None)


def findMatch(fname, out):
    global kp1, des1;
    imgS = cv.imread(fname) # trainImage
    if imgS is None:
        return False
    img2 = cv.cvtColor(imgS, cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp2, des2 = sift.detectAndCompute(img2,None)
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv.FlannBasedMatcher(index_params, search_params)
 
    matches = flann.knnMatch(des1,des2,k=2)
 
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.6*n.distance:
            good.append(m)

    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

        with np.load('CameraParams.npz') as file:
            mtx, dist, _, _ = [file[i] for i in ('cameraMatrix', 'dist', 'rvecs', 'tvecs')]
 
        M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        #drawCountour(img, M)
 
        h,w = img1.shape
        pts = np.float32([ [0,0],[w-1,0], [w-1,h-1],[0,h-1], [w-1, w-1],[0, w-1]]).reshape(-1,1,2)
        dst = cv.perspectiveTransform(pts,M)
        img2 = cv.polylines(imgS,[np.int32(dst[:4, :, :])],True, (0, 255, 255), 6)
        h2, w2 = img2.shape[:2]
        #dist2 = np.zeros(4)
        newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w2,h2), 1, (w2,h2))
        objPoints = np.array([[0., 0., 0.], [w-1, 0., 0.], [w-1, w-1, 0.], [0., w-1, 0.]])
        valid, rvec, tvec = cv.solvePnP(objPoints, dst[[True, True, False, False, True, True]], newcameramtx, dist, cv.SOLVEPNP_P3P)
        #cv.drawFrameAxes(img2, newcameramtx, dist, rvec, tvec, 10000)
        rvecs = cv.Rodrigues(rvec)[0]
        objPoints = np.array([[0., 0., 0.], [0., 0., Z_LENGTH-1],[w-1, 0., 0.], [w-1, 0., Z_LENGTH-1],[w-1, h-1, 0.], [w-1, h-1, Z_LENGTH-1],
                              [0., h-1, 0.], [0., h-1, Z_LENGTH-1]])
        rvecs = cv.Rodrigues(rvec)[0]
        d2 = trans(objPoints, rvecs, tvec, newcameramtx, dist)
        #img2 = cv.polylines(imgS,[np.int32(d2)],True, (0, 0, 255), 2)
        diff_z = np.array(d2[1::2] - d2[::2])
        new_dst = dst[:4, :, :] + diff_z.reshape(-1,1,2)
        img2 = cv.polylines(img2,[np.int32(new_dst)],True, (0, 255, 255), 6)
        col_dst = [dst[0], new_dst[0], new_dst[1], dst[1], dst[2], new_dst[2], new_dst[3], dst[3]]
        img2 = cv.polylines(img2,[np.int32(col_dst)],True, (0, 255, 255), 6)
        for i, m in enumerate(good):
            pt = (int(dst_pts[i][0][0]), int(dst_pts[i][0][1]))
            if matchesMask[i]:
                cv.circle(img2, pt, 5, (0, 255, 255), -1)  # Inliers желтые
            else:
                cv.circle(img2, pt, 5, (0, 0, 255), -1)    # Outliers красные
 
    else:
        print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
        matchesMask = None
    
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
    singlePointColor = None,
    matchesMask = matchesMask, # draw only inliers
    flags = 2)
    #img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
    #plt.imshow(img3),plt.show()
    #img3 = cv.drawKeypoints(img2, [kp2[m.trainIdx] for m in good], cv.DRAW_MATCHES_FLAGS_DEFAULT, color=(0, 255, 255))

    #plt.imshow(img2),plt.show()
    out.write(img2)
    cv.imwrite('./videoframes_tea_detected/frame_'+str(ind)+'.png', img2)
    plt.imshow(img2),plt.show()
    return True

In [ ]:
import glob
ind = 0
img = cv.imread('./videoframes_tea/frame_0.png')
height, width, channels = img.shape
out = cv.VideoWriter('output_tea.mp4', 0, 30.0, (width, height))
while True:
    ret = findMatch('./videoframes_tea/frame_'+str(ind)+'.png', out)
    if not ret:
        break
    ind += 1

out.release()

In [ ]:
out.release()

In [ ]:
img1 = cv.imread('./detect/book.jpg', cv.IMREAD_GRAYSCALE)          # queryImage
h,w = img1.shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
print(pts)

In [ ]:
import glob
ind = 0
img = cv.imread('./videoframes_tea/frame_0.png')
height, width, channels = img.shape 
out = cv.VideoWriter('output.mp4', 0, 30.0, (width, height))
findMatch('./videoframes_tea/frame_500.png', out)

In [ ]:
def drawCountour(img, M):
    h,w = img1.shape
    d = 442
    pts = np.float32([ [0,0,0],[0,h-1,0],[w-1,h-1,0],[w-1,0,0], [0,0,442],[0,h-1,442],[w-1,h-1,442],[w-1,0,442]]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts,M)
    print(dst)
    img2 = cv.polylines(img,[np.int32(dst)],True, (0, 255, 0), 5)
    for i, m in enumerate(good):
        pt = (int(dst_pts[i][0][0]), int(dst_pts[i][0][1]))
        if matchesMask[i]:
            cv.circle(img2, pt, 5, (0, 255, 255), -1)  # Inliers желтые
        else:
            cv.circle(img2, pt, 5, (0, 0, 255), -1)    # Outliers красные